In [8]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import numpy as np
try:
    import mxnet as mx
except:
    sys.stderr.write("Cannot import mxnet.\n")
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.ndimage.interpolation import map_coordinates
from skimage.exposure import rescale_intensity
from skimage.transform import rotate

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *
from distributed_utilities import *
from visualization_utilities import *

Setting environment for Precision WorkStation
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD585/MD585_prep2_thumbnail/MD585-N16-2015.07.16-20.32.33_MD585_2_0047_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD589/MD589_prep2_thumbnail/MD589-N16-2015.07.30-17.03.43_MD589_3_0048_prep2_thumbnail.tif


No vtk
No vtkNot using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.


/media/yuncong/BstemAtlasData/CSHL_data_processed/MD590/MD590_prep2_thumbnail/MD590-IHC17-2015.08.10-19.09.09_MD590_2_0050_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD591/MD591_prep2_thumbnail/MD591-IHC17-2015.08.28-04.23.47_MD591_1_0049_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD592/MD592_prep2_thumbnail/MD592-N16-2015.08.22-00.50.39_MD592_2_0047_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD593/MD593_prep2_thumbnail/MD593-N15-2015.08.21-16.53.02_MD593_3_0045_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD594/MD594_prep2_thumbnail/MD594-IHC16-2015.08.26-16.11.01_MD594_1_0046_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD595/MD595_prep2_thumbnail/MD595-IHC12-2015.09.15-01.21.39_MD595_2_0035_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD598/MD598_prep2_thumbnail/MD598-N18-2015.09.29-17.40.03_MD598_3_0054_prep2_thumbnail.tif
/medi

Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.


/media/yuncong/BstemAtlasData/CSHL_data_processed/MD657/MD657_prep2_thumbnail/MD657-N13-2017.02.22-14.33.34_MD657_3_0039_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD658/MD658_prep2_thumbnail/MD658-N18-2017.03.31-17.34.22_MD658_2_0053_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD661/MD661_prep2_thumbnail/MD662&661-F30-2017.06.05-14.49.48_MD661_1_0088_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F35-2017.06.05-17.08.18_MD662_1_0103_prep2_thumbnail.tif
rm -rf "/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail/CHATM2_slide14_2018_01_18-S3_prep2_thumbnail.tif" && mkdir -p "/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail"
0
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail/CHATM2_slide14_2018_01_18-S3_prep2_thumbnail.tif" "/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2

Not using image_cache.


1
/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_Ntb/CHATM2_slide14_2018_01_18-S3_prep2_thumbnail_Ntb.tif
rm -rf "/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail/CHATM2_slide14_2018_01_18-S3_prep2_thumbnail.tif" && mkdir -p "/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail"
0
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail/CHATM2_slide14_2018_01_18-S3_prep2_thumbnail.tif" "/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail/CHATM2_slide14_2018_01_18-S3_prep2_thumbnail.tif"


Not using image_cache.
Version Ntb is not available. Instead, load lossless and take the blue channel...


1
/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail/CHATM2_slide14_2018_01_18-S3_prep2_thumbnail.tif


Not using image_cache.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Plot histogram over whole images.

In [2]:
stack_versions = {'MD585': 'gray',
'MD589': 'gray',
'MD594': 'gray',
'MD635': 'Ntb',
'MD653': 'Ntb',
 'MD652': 'Ntb',
 'MD642': 'Ntb',
 'MD657': 'Ntb',
 'MD658': 'Ntb',
 'MD661': 'Ntb',
 'MD662': 'Ntb' 
}

In [ ]:
# for stack in ['MD585', 'MD589', 'MD594'] + ['MD635'] + ['MD653', 'MD652', 'MD642', 'MD657', 'MD658'] + ['MD661', 'MD662']:
for stack in ['MD635'] + ['MD653', 'MD652', 'MD642', 'MD657', 'MD658'] + ['MD661', 'MD662']:

    structure = '7N'
    
    if stack in ['MD585', 'MD589', 'MD594'] + ['MD658'] + ['MD635']:
        local_region_bboxes_all_sections = get_local_regions(stack, margin_um=200, by_human=True ,
                                                         structures=['7N_L'])
    elif stack in ['MD653', 'MD652', 'MD642', 'MD657']:
        local_region_bboxes_all_sections = get_local_regions(stack, margin_um=200, by_human=False, level=0.5, 
                                                     structures=['7N_L'],
                                                    detector_id_f=13)
    elif stack in ['MD661', 'MD662']:
        local_region_bboxes_all_sections = get_local_regions(stack, margin_um=200, by_human=False, level=0.5, 
                                                     structures=['7N_L'],
                                                    detector_id_f=1)
    else:
        raise
        
    for name_s, bboxes_all_sections in local_region_bboxes_all_sections.iteritems():
        for sec, bbox in bboxes_all_sections.iteritems():

#     for sec in metadata_cache['valid_sections'][stack]:
        
#             try:
            print sec

            NorF = DataManager.get_brightfield_or_fluorescence(stack=stack, sec=sec)

            lossless_img = DataManager.load_image_v2(stack=stack, prep_id=2, section=sec, version=stack_versions[stack])

            tb_mask = DataManager.load_thumbnail_mask_v3(stack=stack, prep_id=2, section=sec)
            lossless_mask = rescale_by_resampling(tb_mask, 32)

            if NorF == 'N':
                counts, bins = np.histogram(lossless_img[lossless_mask], bins=np.linspace(0, 256, 257));
            elif NorF == 'F':
                counts, bins = np.histogram(lossless_img[lossless_mask], bins=np.linspace(0, 3500, 300));
            else:
                raise

            vmax = lossless_img[lossless_mask].max()
            vmin = lossless_img[lossless_mask].min()

            fig = plt.figure(figsize=(10, 5));
            plt.bar(bins[:-1], counts, align='center')
            if NorF == 'N':
                plt.xlim([-1, 256]);
                plt.xticks(np.r_[bins[::50], vmin, vmax])
            else:
                plt.xlim([-1, 3500]);
                plt.xticks(np.r_[bins[::50], vmin, vmax])
            plt.axvline(x=vmax, c='r');
            plt.axvline(x=vmin, c='r');
            plt.title('Histogram (linear)');
            plt.xlabel('Pixel intensity');
    #                 plt.show();

            plot_fp = os.path.join(ROOT_DIR, 'raw_image_histograms_fullImage', stack, '%s_%d%s.png' % (stack, sec, NorF))
            create_parent_dir_if_not_exists(plot_fp)
            plt.savefig(fname=plot_fp, bbox_inches='tight');
            plt.close(fig);
            upload_to_s3(plot_fp)

            counts_log = np.log(counts)

            fig = plt.figure(figsize=(10, 5));
            plt.bar(bins[:-1], counts_log, width=np.diff(bins)[0]*.8, align='center');
            if NorF == 'N':
                plt.xlim([-1, 256]);
                plt.xticks(np.r_[bins[::50], vmin, vmax])
            else:
                plt.xlim([-1, 3500]);
                plt.xticks(np.r_[bins[::50], vmin, vmax])

            plt.axvline(x=vmax, c='r');
            plt.axvline(x=vmin, c='r');
            plt.title('Histogram (log)');
            plt.xlabel('Pixel intensity');
    #                 plt.show();

            plot_fp = os.path.join(ROOT_DIR, 'raw_image_histograms_log_fullImage', stack, '%s_%d%s.png' % (stack, sec, NorF))
            create_parent_dir_if_not_exists(plot_fp)
            plt.savefig(fname=plot_fp, bbox_inches='tight');
            plt.close(fig);
            upload_to_s3(plot_fp)
#             except:
#                 pass

In [23]:
stack_versions = {'CHATM2': 'Ntb'}

In [24]:
for stack in ['CHATM2']:

    for sec in metadata_cache['valid_sections'][stack][::10]:
        
        try:
        
            print sec

            NorF = DataManager.get_brightfield_or_fluorescence(stack=stack, sec=sec)

            lossless_img = DataManager.load_image_v2(stack=stack, prep_id=2, section=sec, version=stack_versions[stack])

            tb_mask = DataManager.load_thumbnail_mask_v3(stack=stack, prep_id=2, section=sec)
            lossless_mask = rescale_by_resampling(tb_mask, 32)

            if NorF == 'N':
                counts, bins = np.histogram(lossless_img[lossless_mask], bins=np.linspace(0, 256, 257));
            elif NorF == 'F':
                counts, bins = np.histogram(lossless_img[lossless_mask], bins=np.linspace(0, 3500, 300));
            else:
                raise

            vmax = lossless_img[lossless_mask].max()
            vmin = lossless_img[lossless_mask].min()

            fig = plt.figure(figsize=(10, 5));
            plt.bar(bins[:-1], counts, align='center')
            if NorF == 'N':
                plt.xlim([-1, 256]);
                plt.xticks(np.r_[bins[::50], vmin, vmax])
            else:
                plt.xlim([-1, 3500]);
                plt.xticks(np.r_[bins[::50], vmin, vmax])
            plt.axvline(x=vmax, c='r');
            plt.axvline(x=vmin, c='r');
            plt.title('Histogram (linear)');
            plt.xlabel('Pixel intensity');
    #                 plt.show();

            plot_fp = os.path.join(ROOT_DIR, 'raw_image_histograms_fullImage', stack, '%s_%d%s.png' % (stack, sec, NorF))
            create_parent_dir_if_not_exists(plot_fp)
            plt.savefig(fname=plot_fp, bbox_inches='tight');
            plt.close(fig);
            upload_to_s3(plot_fp)

            counts_log = np.log(counts)

            fig = plt.figure(figsize=(10, 5));
            plt.bar(bins[:-1], counts_log, width=np.diff(bins)[0]*.8, align='center');
            if NorF == 'N':
                plt.xlim([-1, 256]);
                plt.xticks(np.r_[bins[::50], vmin, vmax])
            else:
                plt.xlim([-1, 3500]);
                plt.xticks(np.r_[bins[::50], vmin, vmax])

            plt.axvline(x=vmax, c='r');
            plt.axvline(x=vmin, c='r');
            plt.title('Histogram (log)');
            plt.xlabel('Pixel intensity');
    #                 plt.show();

            plot_fp = os.path.join(ROOT_DIR, 'raw_image_histograms_log_fullImage', stack, '%s_%d%s.png' % (stack, sec, NorF))
            create_parent_dir_if_not_exists(plot_fp)
            plt.savefig(fname=plot_fp, bbox_inches='tight');
            plt.close(fig);
            upload_to_s3(plot_fp)
        except Exception as e:
            print e
            pass

42


Not using image_cache.


/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_lossless_Ntb/CHATM2_slide14_2018_01_18-S3_prep2_lossless_Ntb.tif
rm -rf "/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide14_2018_01_18-S3_prep2_thumbnail_mask.png" && mkdir -p "/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask"
0
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide14_2018_01_18-S3_prep2_thumbnail_mask.png" "/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide14_2018_01_18-S3_prep2_thumbnail_mask.png"
1
[Errno 2] No such file or directory: '/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide14_2018_01_18-S3_prep2_thumbnail_mask.png'
52
/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_lossless_Ntb/CHATM2_slide18_2018_01_18-S1_prep2_lossless_Ntb.tif
rm -rf "/m

Not using image_cache.
Version Ntb is not available. Instead, load lossless and take the blue channel...


1
/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_lossless/CHATM2_slide18_2018_01_18-S1_prep2_lossless.tif
Image loading failed.
62


...

/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_lossless_Ntb/CHATM2_slide86_2018_02_13-S2_prep2_lossless_Ntb.tif
rm -rf "/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide86_2018_02_13-S2_prep2_thumbnail_mask.png" && mkdir -p "/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask"
0
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide86_2018_02_13-S2_prep2_thumbnail_mask.png" "/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide86_2018_02_13-S2_prep2_thumbnail_mask.png"
1
[Errno 2] No such file or directory: '/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide86_2018_02_13-S2_prep2_thumbnail_mask.png'
275


Not using image_cache.


/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_lossless_Ntb/CHATM2_slide92_2018_02_13-S3_prep2_lossless_Ntb.tif
rm -rf "/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide92_2018_02_13-S3_prep2_thumbnail_mask.png" && mkdir -p "/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask"
0
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide92_2018_02_13-S3_prep2_thumbnail_mask.png" "/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide92_2018_02_13-S3_prep2_thumbnail_mask.png"
1
[Errno 2] No such file or directory: '/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide92_2018_02_13-S3_prep2_thumbnail_mask.png'
295


Not using image_cache.


/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_lossless_Ntb/CHATM2_slide100_2018_02_13-S1_prep2_lossless_Ntb.tif
rm -rf "/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide100_2018_02_13-S1_prep2_thumbnail_mask.png" && mkdir -p "/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask"
0
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide100_2018_02_13-S1_prep2_thumbnail_mask.png" "/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide100_2018_02_13-S1_prep2_thumbnail_mask.png"
1
[Errno 2] No such file or directory: '/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_thumbnail_mask/CHATM2_slide100_2018_02_13-S1_prep2_thumbnail_mask.png'
